## Building a Predictive Model with Snowpark ML

## 1. Import the following Libraries: (Use the Packages option above)

i) snowflake-ml-python

ii) pandas

In [3]:
# Import of Libraries
import warnings

import pandas as pd
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.metrics import accuracy_score
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.preprocessing import LabelEncoder
from snowflake.ml.modeling.ensemble import RandomForestClassifier
# from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session
from snowflake.snowpark import types as T
from snowflake.snowpark.functions import col

warnings.simplefilter(action="ignore", category=UserWarning)

## 2. Importing the Snowflake ML Registry

Loads the Registry module, which enables storing, managing, and retrieving trained ML models in Snowflake.

In [4]:
from snowflake.ml.registry import Registry
# from snowflake.ml.registry import model_registry

## 3. Getting the Active Snowflake Session

Establishes an active Snowflake session, which is required for executing Snowpark operations.

In [6]:
session = get_active_session()

## 4. Loading Loan Data from Snowflake Table

Retrieves data from the LOAN_DATA table in Snowflake and loads it into a Snowpark DataFrame. The goal is to predict whether loan could be approved for a customer or not. The show() function displays a sample of the dataset.

In [7]:
loan_df = session.table("LOAN_DATA")
loan_df.show()

----------------------------------------------------------------------------------------------------------------------------------------
|"NAME"   |"MARITAL_STATUS"  |"DAYS_ACT_OPEN"  |"AGE"  |"INCOME"  |"ANY_PREVIOUS_DEFAULT"  |"GENDER"  |"OCCUPATION"  |"LOAN_APPROVAL"  |
----------------------------------------------------------------------------------------------------------------------------------------
|John     |Single            |2058             |28     |3000      |False                   |M         |Engineer      |1                |
|Mary     |Married           |2487             |34     |5000      |True                    |F         |Teacher       |0                |
|David    |Single            |1577             |22     |2000      |False                   |M         |Doctor        |1                |
|Sarah    |Married           |2896             |40     |8000      |False                   |F         |Engineer      |1                |
|Mike     |Single            |2349       

## 5. Separating Categorical and Continuous Variables

Group the categorical and continuous feature columns separately to facilitate preprocessing.

In [10]:
cat_cols = ['GENDER', 'MARITAL_STATUS','OCCUPATION']
cont_cols = ['DAYS_ACT_OPEN', 'AGE', 'INCOME']

## 6. Handling Missing Values using Imputation

Uses SimpleImputer to replace missing values in categorical columns with the most frequently occurring value.

In [11]:
impute_cat = SimpleImputer(
    input_cols=cat_cols,
    output_cols=cat_cols,
    strategy="most_frequent",
    drop_input_cols=True,
)

loan_df = impute_cat.fit(loan_df).transform(loan_df)
loan_df.show()

Input value type doesn't match the target column data type, this replacement was skipped. Column Name: "ANY_PREVIOUS_DEFAULT", Type: StringType(16777216), Input Value: False, Type: <class 'bool'>


----------------------------------------------------------------------------------------------------------------------------------------
|"GENDER"  |"MARITAL_STATUS"  |"ANY_PREVIOUS_DEFAULT"  |"OCCUPATION"  |"NAME"   |"DAYS_ACT_OPEN"  |"AGE"  |"INCOME"  |"LOAN_APPROVAL"  |
----------------------------------------------------------------------------------------------------------------------------------------
|M         |Single            |false                   |Engineer      |John     |2058             |28     |3000      |1                |
|F         |Married           |true                    |Teacher       |Mary     |2487             |34     |5000      |0                |
|M         |Single            |false                   |Doctor        |David    |1577             |22     |2000      |1                |
|F         |Married           |false                   |Engineer      |Sarah    |2896             |40     |8000      |1                |
|M         |Single            |true      

## 7. Label Encoding Categorical Variables

Converts categorical values into numerical representations using label encoding.

In [12]:
# Label Encoding
for i in cat_cols:
    LE = LabelEncoder(
    input_cols=i,
    output_cols=f"{i}_ENCODED",
    drop_input_cols=True)
    loan_df = LE.fit(loan_df).transform(loan_df)
    
   



----------------------------------------------------------------------------------------------------------------------------------------
|"OCCUPATION"  |"ANY_PREVIOUS_DEFAULT"  |"MARITAL_STATUS"  |"GENDER"  |"NAME"   |"DAYS_ACT_OPEN"  |"AGE"  |"INCOME"  |"LOAN_APPROVAL"  |
----------------------------------------------------------------------------------------------------------------------------------------
|2.0           |0.0                     |1.0               |1.0       |John     |2058             |28     |3000      |1                |
|5.0           |1.0                     |0.0               |0.0       |Mary     |2487             |34     |5000      |0                |
|1.0           |0.0                     |1.0               |1.0       |David    |1577             |22     |2000      |1                |
|2.0           |0.0                     |0.0               |0.0       |Sarah    |2896             |40     |8000      |1                |
|0.0           |1.0                     |

In [ ]:

loan_df.show()

## 8. Dropping Unused Columns

Remove the NAME column, as it is irrelevant for predictive modeling.

In [13]:
loan_df = loan_df.drop("NAME")

## 9. Splitting Data into Training and Testing Sets

Split the dataset into an 80% training set and a 20% test set, using a fixed seed for reproducibility.

In [14]:
train_df, test_df = loan_df.random_split(weights=[0.8, 0.2], seed=8)

## 10. Initializing a Random Forest Classifier and fitting it on training data
 
Create a RandomForestClassifier model and define input features, target label, and output column. The model is then fitted against the training dataset

In [15]:
# Random forest model
regressor = RandomForestClassifier(
    input_cols=train_df.drop("LOAN_APPROVAL").columns,
    label_cols="LOAN_APPROVAL",
    output_cols="PRED_APPROVED"
)
regressor.fit(train_df)

## 11. Setting up the Model Registry

To use a trained machine learning model for predictions within Snowflake, you must first register it in the Snowflake Model Registry. This secure repository allows you to manage your models and their associated information within Snowflake, regardless of the model's origin or type. Once registered, running inference on the model becomes straightforward.
The command below initializes a Snowflake ML model registry and prepares it to log the trained model.

In [17]:
# Create a registry to log the model
reg = Registry(session=session, database_name='SNOWPARK_ML_DEMO', 
               schema_name='PUBLIC' )

# Model Registry

## 12. Logging the Model into Registry
Store the trained model in the Snowflake model registry as version V1.

In [ ]:
# Logging our model in the Registry

# Define model name and version (use uppercase for name)
model_name = "LOAN"
model_version = 'V1'

# Get sample input data to pass into the registry logging function
X = train_df.drop("LOAN_APPROVAL")

# Let's first log the very first model we trained
model_ver = reg.log_model(
    model_name=model_name,
    version_name=model_version,
    model=regressor,
    sample_input_data=X, # to provide the feature schema
)

## 13. Listing Registered Models
Display all models stored in the model registry.



In [18]:
# List model
reg.show_models()

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-03-06 00:02:15.909000-08:00,LOAN,LOAN,DEPARTMENT,None,ACCOUNTADMIN,V0,"[""V0""]"


## 14. Listing Model Versions
Retrieve and display the different versions of the LOAN model.We have just the one version (V1)

In [31]:
reg.get_model(model_name).show_versions()

,created_on,name,comment,database_name,schema_name,module_name,is_default_version,functions,metadata,user_data
0,2024-03-06 00:02:15.942000-08:00,V0,None,LOAN,DEPARTMENT,LOAN,true,"[""PREDICT_PROBA"",""PREDICT"",""PREDICT_LOG_PROBA""]",{},"{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."
1,2024-03-06 00:22:13.240000-08:00,V1,None,LOAN,DEPARTMENT,LOAN,false,"[""PREDICT_PROBA"",""PREDICT"",""PREDICT_LOG_PROBA""]","{""metrics"": {""accuracy"": 1.0}, ""snowpark_ml_sc...","{""snowpark_ml_data"":{""functions"":[{""name"":""PRE..."


## 15. Setting a Default Model Version
Assign V1 as the default model version for deployment and predictions.

In [32]:
m = reg.get_model(model_name)
m.default = 'V1'
mod_v = m.default

## 16. Running Predictions on Test Data
Use the registered model to generate predictions for loan approval on the test dataset.

In [39]:
# Predictions 
test_prediction = mod_v.run(test_df, function_name="predict")
test_prediction.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"LOAN_APPROVAL"  |"OCCUPATION"  |"ANY_PREVIOUS_DEFAULT"  |"MARITAL_STATUS"  |"GENDER"  |"DAYS_ACT_OPEN"  |"AGE"  |"INCOME"  |"PREDICT_PROBA_0"     |"PREDICT_PROBA_1"    |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                |4.0           |0.0                     |0.0               |0.0       |2118             |29     |3500      |0.05133735708735709   |0.9486626429126428   |
|0                |2.0           |1.0                     |1.0               |1.0       |3400             |47     |6000      |0.8580779220779223    |0.14192207792207795  |
|1                |3.0           |0.0                     |0.0               |0.0       |2314             |32     |4500      |0.069713439338

## 17. Verify Model Accuracy

In [ ]:
from snowflake.ml.modeling.metrics import accuracy_score
# Accuracy of model
accuracy = accuracy_score(
    df=test_prediction, y_true_col_names="LOAN_APPROVAL", y_pred_col_names="PRED_APPROVED"
)

print(f"Accuracy: {accuracy}")

## 18. Storing Test data in a Snowflake table

In [39]:
test_df.write.mode("overwrite").save_as_table("LOAN_TEST_DATA")

## 19. Querying the Test data table

In [ ]:
loan_df_test = session.table("LOAN_TEST_DATA")
loan_df_test.show()